In [1]:
import glob
dirs = glob.glob("/mnt/bn/nlhei-nas/liubangya/proj/vlm/workspace/image_*")
import numpy as np
import json

def dictify(s):
    s = s.split(", ")
    ret_dict = {}
    for ss in s:
        triplet = ss.split(' ')
        if len(triplet) != 3:
            return None
        try:
            triplet = [int(triplet[0]), int(triplet[1]), float(triplet[2])]
        except:
            return None
        ret_dict[triplet[0]] = triplet[1:]
    return ret_dict


MAX_ERROR_PER_FRAME = 5

def eval(ans_path):
    with open(ans_path, "r") as f:
        lines = f.readlines()
    total_error = 0
    per_frame_stat = {}
    for l in lines:
        data = json.loads(l)
        gt = data["gt_ans"]
        pred = data["ans"]
        gt = dictify(gt)
        pred = dictify(pred)

        error_per_ans = 0
        if pred is None:
            error_per_ans = 1
            total_error += error_per_ans
            continue

        for k in gt.keys():
            if k not in pred:
                error_per_ans += 1
                per_frame_stat.setdefault(k, []).append(1)
            else:
                dist = (pred[k][0] - gt[k][0])**2 + (pred[k][1] - gt[k][1])**2
                dist = min(dist**0.5, MAX_ERROR_PER_FRAME)
                error_per_ans += dist / MAX_ERROR_PER_FRAME
                per_frame_stat.setdefault(k, []).append(dist / MAX_ERROR_PER_FRAME)
        
        error_per_ans /= len(gt)
        total_error += error_per_ans
    
    per_frame_stat = {k: float(np.mean(v)) * MAX_ERROR_PER_FRAME for k, v in per_frame_stat.items()}
    return total_error, per_frame_stat




In [2]:
import glob
import os

for d in dirs:
    ans = glob.glob(
        os.path.join(d, "results/*.json")
    )
    for a in ans:
        exp_name = os.path.basename(d)
        setup_name = os.path.basename(a).replace(".json", "")
        score, score_per_frame = eval(a)
        print(f"""
        {exp_name} {setup_name} >>>
        {score}
        {score_per_frame}
        """)


        image_scannetpp_new_16x24_2 ans_base >>>
        1434.708449300955
        {0: 0.08837890625, 1: 4.710002193852343}
        



        image_scannetpp_new_16x24_2 ans_lora >>>
        464.9157493456624
        {0: 0.0, 1: 1.8596629973826468}
        

        image_scannetpp_new_16x24_5 ans_lora >>>
        619.1838515055274
        {0: 0.0, 1: 1.2939205670996072, 2: 1.6532835532312755, 3: 1.9244553388833316, 4: 2.077198045199322}
        
